# 서베이 데이터 효율적으로 분석하며 인사이트 도출하기

- 전반적인 분석 사례 과정을 설명하기 위해 임시로 만든 데이터셋입니다.
- 실제 사용된 데이터셋과는 다릅니다.

In [1]:
import pandas as pd
import numpy as np
pd.set_option('mode.chained_assignment', None)

# Load Dataset

In [2]:
# Raw Data
df_raw = pd.read_excel('./survey_sample_fastcampus.xlsx')
df_raw

,ID,성별,나이,사용하는앱,최근사용한앱,사용빈도,사용이유,만나고싶은사람,소통형태
0,1,여,14세 이상-18세 미만,스푼,스푼,거의 매일 이용,바나나,축구선수,C형
1,2,여,30-39세,C사,C사,주 3-5회,포도,야구선수,B형
2,3,여,18-24세,"스푼, A사, B사, C사",스푼,주 3-5회,기타,"축구선수, 야구선수, 테니스선수, 수영선수",C형
3,4,남,25-29세,"스푼, C사",C사,거의 매일 이용,포도,"축구선수, 테니스선수",B형
4,5,남,40세 이상,"B사, E사",E사,주 2회 이하,오렌지,수영선수,B형
...,...,...,...,...,...,...,...,...,...
2949,2950,여,18-24세,스푼,스푼,거의 매일 이용,수박,"축구선수, 야구선수, 테니스선수, 수영선수",C형
2950,2951,남,18-24세,스푼,스푼,주 3-5회,사과,"축구선수, 야구선수, 테니스선수",C형
2951,2952,남,25-29세,스푼,스푼,주 3-5회,오렌지,"축구선수, 테니스선수",B형
2952,2953,여,25-29세,스푼,스푼,거의 매일 이용,사과,"축구선수, 테니스선수, 수영선수",B형


In [3]:
# null값 = 응답없음으로 처리
df_raw.fillna('응답없음', inplace=True)

In [4]:
df_raw.dtypes

ID          int64
성별         object
나이         object
사용하는앱      object
최근사용한앱     object
사용빈도       object
사용이유       object
만나고싶은사람    object
소통형태       object
dtype: object

In [5]:
# Raw Data
df_mapper = pd.read_excel('/Users/justin/Desktop/fastcampus/survey_mapper_fastcampus.xlsx')
print(df_mapper.shape)
print(df_mapper.head())

(42, 7)
  q_id         original_q column_nm          label  scale multi_yn  note
0    1     귀하의 성별은 무엇입니까?        성별              남      0        n   NaN
1    1     귀하의 성별은 무엇입니까?        성별              여      1        n   NaN
2    2  귀하의 연령은 어떻게 되십니까?        나이  14세 이상-18세 미만      0        n   NaN
3    2  귀하의 연령은 어떻게 되십니까?        나이         18-24세      1        n   NaN
4    2  귀하의 연령은 어떻게 되십니까?        나이         25-29세      2        n   NaN


# Understanding Data

In [6]:
# Check unique values
# single_response
print(df_raw['성별'].value_counts())
print(round(df_raw['성별'].value_counts(normalize=True)*100,2))

여    1600
남    1354
Name: 성별, dtype: int64
여    54.16
남    45.84
Name: 성별, dtype: float64


In [7]:
# multi_response
tmp_list = []

for v in df_raw['만나고싶은사람']:
    tmp_list.extend(v.split(', '))

print(pd.Series(tmp_list).value_counts())
print(round(pd.Series(tmp_list).value_counts(normalize=True)*100,2))

축구선수         2040
수영선수         1200
테니스선수        1008
야구선수          384
기타_직접입력15       2
기타_직접입력20       2
기타_직접입력30       2
기타_직접입력35       2
기타_직접입력5        2
기타_직접입력7        2
기타_직접입력26       2
기타_직접입력21       2
기타_직접입력14       2
기타_직접입력13       2
기타_직접입력12       2
기타_직접입력29       2
기타_직접입력34       2
기타_직접입력33       2
기타_직접입력19       2
기타_직접입력2        2
기타_직접입력32       2
기타_직접입력16       2
기타_직접입력11       2
기타_직접입력28       2
기타_직접입력18       2
기타_직접입력24       2
기타_직접입력22       2
기타_직접입력3        2
기타_직접입력31       2
기타_직접입력17       2
기타_직접입력27       2
기타_직접입력23       2
기타_직접입력9        2
기타_직접입력4        2
기타_직접입력25       2
기타_직접입력8        2
기타_직접입력10       2
기타_직접입력1        2
기타_직접입력6        2
dtype: int64
축구선수         43.39
수영선수         25.52
테니스선수        21.44
야구선수          8.17
기타_직접입력15     0.04
기타_직접입력20     0.04
기타_직접입력30     0.04
기타_직접입력35     0.04
기타_직접입력5      0.04
기타_직접입력7      0.04
기타_직접입력26     0.04
기타_직접입력21     0.04
기타_직접입력14     0.04
기타_직접입력13     0.04
기타_직접입력12     0.04


# Create Definitions

## General

### Sort index

In [8]:
# Sort index
def def_sort_index(df,col,axis,level):
    sorted_labels = df_mapper[df_mapper['column_nm'] == col].sort_values(by='scale')['label'].tolist()
    output_list = []
    cols = []
    if axis == 1:
        if len(df.columns.names) == 1:
            for v in df.columns.tolist():
                if v not in cols:
                    cols.append(v) 
            for v in sorted_labels + cols:
                if v not in output_list:
                    output_list.append(v)
            df = df.reindex(output_list, axis=1, fill_value=0)            
        elif len(df.columns.names) == 2:
            for v in df.columns:
                v = v[level]
                if v not in cols:
                    cols.append(v) 
            for v in sorted_labels + cols:
                if v not in output_list:
                    output_list.append(v)
            df = df.reindex(output_list, axis=1, level=level, fill_value=0)
        
    # Sorting Index
    elif axis==None:
        for v in sorted_labels + df.index.tolist():
            if v not in output_list:
                output_list.append(v)
                if v not in df.index.tolist():
                    df_tmp = pd.DataFrame(data=[np.zeros(df.shape[1])], columns=df.columns.tolist()).rename(index={0: v})
                    df = df.append(df_tmp)
        df = df.reindex(output_list, axis=0, fill_value=0)

    return df

### Calculate Total

In [9]:
def def_get_total(df, multi_yn, iv):
    if multi_yn == 'n':
        df['Total'] = df.sum(axis=1, skipna = True).values
        row_total = pd.DataFrame(df.sum(axis = 0, skipna = True)).T
        row_total.rename(index={0: 'Total'}, inplace=True)
        df = pd.concat([df, row_total], axis=0)

    elif (multi_yn == 'y') & (len(df.columns.names) == 1):
        df_vcounts = pd.DataFrame(df_raw[iv[0]].value_counts()).T.rename(index={iv[0]:'Total'})
        df_vcounts = def_sort_index(df_vcounts,iv[0],1,0)
        df = df.append(df_vcounts)
        df['Total'] = df.sum(axis=1, skipna = True).values
        
    elif (multi_yn == 'y') & (len(df.columns.names) == 2):
        df_vcounts = pd.DataFrame(df_raw[iv].value_counts()).T
        for e, v in enumerate(iv):
            df_vcounts = def_sort_index(df_vcounts,iv[e],1,e)
            df_vcounts.rename(index={0:'Total'}, inplace=True)
        df = df.append(df_vcounts)
        df['Total'] = df.sum(axis=1, skipna = True).values
        
    return df

### Calculate Percentage

In [10]:
def def_get_percentage(df):
    df_per = df.copy()
    for i in range(0,len(df_per.columns)):
        df_per.iloc[:,i] = round(df.iloc[:,i] / df.iloc[-1,i]*100,2)
    df_per.fillna(0, inplace=True)

    # concat % character
    for percol in df_per.columns:
        df_per[percol] = [str(v)+"%" for v in df_per[percol]]
        
    return df_per

## Cross Table

### 단일 X 단일응답

In [11]:
def s_answers_pivot(df, iv, dv):
    cols = list()
    cols.extend(iv)
    cols.append(dv)

    df_temp = df[cols].copy()

    # Frequency & Sorting Header
    if len(iv) == 1:
        df_crosstab = pd.crosstab(df_temp[iv[0]], df_temp[dv]).T
        df_crosstab = def_sort_index(df_crosstab,iv[0],1,0)
    elif len(iv) == 2:
        df_crosstab = pd.crosstab([df_temp[iv[0]], df_temp[iv[1]]], df_temp[dv]).T
        for i, col in enumerate(iv):
            df_crosstab = def_sort_index(df_crosstab,col,1,i)
    else:
        print('Exceeded number of Input variables')

    # Sorting DV
    df_crosstab = def_sort_index(df_crosstab,dv,None,None)

    # Get Total 
    df_freq = def_get_total(df_crosstab, 'n', iv)
    
    # Percentage table
    df_per = def_get_percentage(df_freq)
    
    return df_freq, df_per

### 단일 X 복수응답

In [12]:
def m_answers_pivot(df, iv, dv):
    cols = list()
    cols.extend(iv)
    cols.append(dv)

    df_temp = df[cols].copy()

    # DV 컬럼 마지막에 comma 붙이기
    df_temp[dv] = [str(v)+',' for v in df_temp[dv]]

    # Mapper
    mapper = df_mapper[df_mapper['column_nm'] == dv]['label'].tolist()
    mappers_comma = [v+',' for v in mapper]

    # label마다 컬럼만들기 문자가 포함되면 1 그렇지 않으면 0
    for label in mappers_comma:
        df_temp[label[:-1]] = [int(label in v) for v in df_temp[dv]]

    # Others 처리 label이외에 어떤 문자가 적혀있으면 1 그렇지 않으면 0
    others_list = []
    for v in df_temp[dv]:
        for label in mappers_comma:
            v = v.replace(label,'').replace('nan','')
        others_list.append(int(bool(v.strip())))
    df_temp['기타(직접 입력)'] = others_list

    # 원래 dv컬럼 제거
    del df_temp[dv]
    dv_cols = df_temp.columns.tolist()
    dv_cols = [v for v in dv_cols if v not in iv]

    # Crosstab(Frequency)
    df_freq = df_temp.groupby(by=iv)[dv_cols].sum().T
    if len(iv) == 1:
        df_freq = def_sort_index(df_freq,iv[0],1,0)
    elif len(iv) == 2:
        for i, col in enumerate(iv):
            df_freq = def_sort_index(df_freq,col,1,i)
    
    # Get Total 
    df_freq = def_get_total(df_freq, 'y', iv)
    
    # Percentage table 
    df_per = def_get_percentage(df_freq)

    return df_freq, df_per

In [13]:
# iv = ['나이']
# dv = '최근사용한앱'

# iv = ['성별','나이']
# dv = '최근사용한앱'

# iv = ['최근사용한앱']
# dv = '나이'

iv = ['성별','최근사용한앱']
dv = '나이'

df_freq, df_per = s_answers_pivot(df_raw, iv, dv)

# Unload

In [14]:
import os
path = '/Users/justin/Desktop/fastcampus/result/'

## General

In [15]:
iv_list = ['성별', '나이', '최근사용한앱', '사용빈도', '소통형태']
dv_list = ['성별', '나이', '사용하는앱', '최근사용한앱', '사용빈도', '사용이유', '만나고싶은사람', '소통형태']
unload_path = path+'rt_general.xlsx'

for i in iv_list:
    for dv in dv_list:
        if i != dv:
            iv = []
            iv.append(i)            
            if df_mapper[df_mapper['column_nm'] == dv]['multi_yn'].values[0] == 'n':
                df_freq, df_per = s_answers_pivot(df_raw,iv,dv)
            else:
                df_freq, df_per = m_answers_pivot(df_raw,iv,dv)            

            sheet_name = iv[0]+'X'+dv
            if not os.path.exists(unload_path):
                with pd.ExcelWriter(unload_path, mode='w', engine='openpyxl') as writer:
                    df_freq.to_excel(writer, sheet_name = sheet_name)
                    df_per.to_excel(writer, sheet_name = sheet_name, startrow=df_freq.shape[0]+2, startcol=0)
            else:
                with pd.ExcelWriter(unload_path, mode='a', engine='openpyxl') as writer:
                    df_freq.to_excel(writer, sheet_name = sheet_name)
                    df_per.to_excel(writer, sheet_name = sheet_name, startrow=df_freq.shape[0]+2, startcol=0)
            print(sheet_name)

성별X나이
성별X사용하는앱
성별X최근사용한앱
성별X사용빈도
성별X사용이유
성별X만나고싶은사람
성별X소통형태
나이X성별
나이X사용하는앱
나이X최근사용한앱
나이X사용빈도
나이X사용이유
나이X만나고싶은사람
나이X소통형태
최근사용한앱X성별
최근사용한앱X나이
최근사용한앱X사용하는앱
최근사용한앱X사용빈도
최근사용한앱X사용이유
최근사용한앱X만나고싶은사람
최근사용한앱X소통형태
사용빈도X성별
사용빈도X나이
사용빈도X사용하는앱
사용빈도X최근사용한앱
사용빈도X사용이유
사용빈도X만나고싶은사람
사용빈도X소통형태
소통형태X성별
소통형태X나이
소통형태X사용하는앱
소통형태X최근사용한앱
소통형태X사용빈도
소통형태X사용이유
소통형태X만나고싶은사람


## Demographic

In [16]:
iv = ['성별','나이']
dv_list = ['사용하는앱','최근사용한앱','사용빈도','사용이유','만나고싶은사람','소통형태']
unload_path = path+'rt_demographic.xlsx'

for dv in dv_list:
    if df_mapper[df_mapper['column_nm'] == dv]['multi_yn'].values[0] == 'n':
        df_freq, df_per = s_answers_pivot(df_raw,iv,dv)
    else:
        df_freq, df_per = m_answers_pivot(df_raw,iv,dv)            

    sheet_name = dv
    if not os.path.exists(unload_path):
        with pd.ExcelWriter(unload_path, mode='w', engine='openpyxl') as writer:
            df_freq.to_excel(writer, sheet_name = sheet_name)
            df_per.to_excel(writer, sheet_name = sheet_name, startrow=df_freq.shape[0]+5, startcol=0)
    else:
        with pd.ExcelWriter(unload_path, mode='a', engine='openpyxl') as writer:
            df_freq.to_excel(writer, sheet_name = sheet_name)
            df_per.to_excel(writer, sheet_name = sheet_name, startrow=df_freq.shape[0]+5, startcol=0)
    print(sheet_name)

사용하는앱
최근사용한앱
사용빈도
사용이유
만나고싶은사람
소통형태


## Spoon

In [17]:
# Sperate whether Spoon is the most used app in a recent month
df_raw['스푼사용여부'] = ['스푼 사용' if (('스푼' == main) & (sub != '현재 이용하지 않음')) else '스푼 미사용' for main, sub in zip(df_raw['최근사용한앱'], df_raw['사용빈도'])]
df_raw['스푼사용여부'].value_counts()

스푼 사용     2086
스푼 미사용     868
Name: 스푼사용여부, dtype: int64

In [18]:
# General
iv = ['스푼사용여부']
dv_list = ['사용하는앱','최근사용한앱','사용빈도','사용이유','만나고싶은사람','소통형태']
unload_path = path+'rt_spoonyn.xlsx'

for dv in dv_list:
    if df_mapper[df_mapper['column_nm'] == dv]['multi_yn'].values[0] == 'n':
        df_freq, df_per = s_answers_pivot(df_raw,iv,dv)
    else:
        df_freq, df_per = m_answers_pivot(df_raw,iv,dv)
    
    sheet_name = dv
    if not os.path.exists(unload_path):
        with pd.ExcelWriter(unload_path, mode='w', engine='openpyxl') as writer:
            df_freq.to_excel(writer, sheet_name = sheet_name)
            df_per.to_excel(writer, sheet_name = sheet_name, startrow=df_freq.shape[0]+2, startcol=0)            
    else:
        with pd.ExcelWriter(unload_path, mode='a', engine='openpyxl') as writer:
            df_freq.to_excel(writer, sheet_name = sheet_name)
            df_per.to_excel(writer, sheet_name = sheet_name, startrow=df_freq.shape[0]+2, startcol=0)            
    print(sheet_name)

사용하는앱
최근사용한앱
사용빈도
사용이유
만나고싶은사람
소통형태


In [19]:
# Demographic
iv = ['스푼사용여부','나이']
dv_list = ['사용하는앱','최근사용한앱','사용빈도','사용이유','만나고싶은사람','소통형태']
unload_path = path+'rt_spoonyn_demo.xlsx'

for dv in dv_list:
    if df_mapper[df_mapper['column_nm'] == dv]['multi_yn'].values[0] == 'n':
        df_freq, df_per = s_answers_pivot(df_raw,iv,dv)
    else:
        df_freq, df_per = m_answers_pivot(df_raw,iv,dv)
    

    sheet_name = dv
    if not os.path.exists(unload_path):
        with pd.ExcelWriter(unload_path, mode='w', engine='openpyxl') as writer:
            df_freq.to_excel(writer, sheet_name = sheet_name)
            df_per.to_excel(writer, sheet_name = sheet_name, startrow=df_freq.shape[0]+5, startcol=0)
    else:
        with pd.ExcelWriter(unload_path, mode='a', engine='openpyxl') as writer:
            df_freq.to_excel(writer, sheet_name = sheet_name)
            df_per.to_excel(writer, sheet_name = sheet_name, startrow=df_freq.shape[0]+5, startcol=0)
    print(sheet_name)

사용하는앱
최근사용한앱
사용빈도
사용이유
만나고싶은사람
소통형태


# Chi-square Testing

- 질적자료 질적자료간에 서로 통계적으로 관계가 있는지 판단하는 검정
- 카이제곱 분포를 기반으로 관찰된 빈도와 기대하는 빈도가 의미있게 다른지 여부를 검정
- 귀무가설: 나이와 사용이유는 관련성이 있다
- 대립가설: 나이와 사용이유는 관련성이 없다

In [20]:
import scipy.stats as stats

In [21]:
df_freq, df_per = s_answers_pivot(df_raw, ['나이'], '사용이유')
df_freq_prc = df_freq.iloc[:-1,:-1]
df_freq_prc

나이,14세 이상-18세 미만,18-24세,25-29세,30-39세,40세 이상
바나나,102,248,94,72,26
포도,50,130,34,18,10
오렌지,16,80,78,58,18
사과,282,646,292,210,48
수박,24,56,30,68,24
배,4,20,14,26,12
기타,36,70,26,30,2


In [22]:
rt = stats.chi2_contingency(observed=df_freq_prc)
chi_square = round(rt[0],4)
pvalue = round(rt[1],4)
print(chi_square, pvalue)

sr_stat = pd.DataFrame([chi_square, pvalue])
sr_stat.rename(index={0:'chi_square', 1:'pvalue'}, inplace=True)
sr_stat.columns = ['stat']
sr_stat

225.2543 0.0


,stat
chi_square,225.2543
pvalue,0.0000
